In [ ]:
!pip install langchain faiss-cpu

In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
from langchain_community.vectorstores import FAISS

In [8]:
def count_paragraphs(corpus):
    # Split the corpus into paragraphs based on newline characters
    paragraphs = corpus.split('\n\n')  # Assuming paragraphs are separated by two newline characters

    # Count the number of paragraphs
    num_paragraphs = len(paragraphs)
    
    return num_paragraphs

In [11]:
def average_paragraph_length(corpus):
    # Split the corpus into paragraphs based on newline characters
    paragraphs = corpus.split('\n\n')  # Assuming paragraphs are separated by two newline characters

    # Calculate the length of each paragraph
    paragraph_lengths = [len(paragraph) for paragraph in paragraphs]

    # Calculate the average length of paragraphs
    if paragraph_lengths:
        average_length = sum(paragraph_lengths) / len(paragraph_lengths)
    else:
        average_length = 0

    return average_length

In [6]:
# read text file
text_corpus = open("D:\Projects\geeta_gpt\docs\geeta_text.txt", "r").read()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
# docs = text_splitter.split_documents(documents)

In [7]:
text_corpus

'\nBHAGAVAD-GITA in ENGLISH\nAuthor: Sage Veda Vyasa\nTranslated in English: Ramananda Prasad, Ph.D.\n\nINTRODUCTION\nThe Bhagavad-Gita is a doctrine of universal truth and a book \nof moral and spiritual growth. Its message is sublime and non-sectarian. It deals with the most sacred metaphysical science. It imparts the knowledge of the Self and answers two universal questions: Who am I, and how can I lead a happy and peaceful life in \nthis word full of dualities and dilemmas?\nIt\'s a timeless book of wisdom that inspired Thoreau, Emerson, \nEinstein, Oppenheimer, Gandhi and many others. The BhagavadGita teaches us how to equip ourselves for the battle of life. A repeated study with faith purifies our psyche and guides us to face \nthe challenges of modern living leading to inner peace and happiness.\nGita teaches the spiritual science of Self-realization (SR) based \non the essence of Upanishads and Vedanta. The prime message \nof the Gita is: The ultimate purpose of life is to real

In [10]:
num_paragraphs = count_paragraphs(text_corpus)
print("Number of paragraphs:", num_paragraphs)

Number of paragraphs: 50


In [12]:
avg_length = average_paragraph_length(text_corpus)
print("Average paragraph length:", avg_length)

Average paragraph length: 2578.56


In [26]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("D:\Projects\geeta_gpt\docs\geeta_text.txt")
loader.load()

[Document(page_content='\nBHAGAVAD-GITA in ENGLISH\nAuthor: Sage Veda Vyasa\nTranslated in English: Ramananda Prasad, Ph.D.\n\nINTRODUCTION\nThe Bhagavad-Gita is a doctrine of universal truth and a book \nof moral and spiritual growth. Its message is sublime and non-sectarian. It deals with the most sacred metaphysical science. It imparts the knowledge of the Self and answers two universal questions: Who am I, and how can I lead a happy and peaceful life in \nthis word full of dualities and dilemmas?\nIt\'s a timeless book of wisdom that inspired Thoreau, Emerson, \nEinstein, Oppenheimer, Gandhi and many others. The BhagavadGita teaches us how to equip ourselves for the battle of life. A repeated study with faith purifies our psyche and guides us to face \nthe challenges of modern living leading to inner peace and happiness.\nGita teaches the spiritual science of Self-realization (SR) based \non the essence of Upanishads and Vedanta. The prime message \nof the Gita is: The ultimate pur

In [27]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=300)
docs = text_splitter.split_documents(loader.load())

In [30]:
len(docs)

153

In [4]:
# loading text embedding model
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
model_name = "BAAI/bge-small-en-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

In [35]:
# Creating vector database
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(docs, hf)
db.save_local("faiss_index")

In [5]:
# creating vector database reterival
new_db = FAISS.load_local("faiss_index", hf)
retriever = new_db.as_retriever(search_kwargs={"k":5})

In [6]:
query = 'how to Sthitpragya'
context=retriever.get_relevant_documents(query)
context

[Document(page_content='senses under control, and becoming free from desires and proprietorship. (6.10) One should sit on his or her own firm seat that is \nneither too high nor too low, covered with dry grass, a deerskin, \nand a cloth, one over the other, in a clean spot. Sitting there in a \ncomfortable position and concentrating the mind on God, controlling the thoughts and the activities of the senses, one should practice meditation for self-purification. (6.11-12) One should sit by \nholding the waist, spine, chest, neck, and head erect, motionless \nand steady; fix the eyes and the mind steadily in front of the nostrils, without looking around; make your mind serene and fearless; \npractice celibacy; have the mind under control; think of Me; and', metadata={'source': 'D:\\Projects\\geeta_gpt\\docs\\geeta_text.txt'}),
 Document(page_content="thoughts are set on My personal form, O Arjuna. (12.06-07)\nThe four paths to God\nTherefore, focus your mind on Me, and let your intellect 

In [59]:
query = 'how to live life fully'
context=retriever.get_relevant_documents(query)
context

[Document(page_content='realization in this very life as if there were no reincarnation. Live as \nthough this is your last day on this earth. One cannot achieve \nanything through laziness and procrastination. \nSeek refuge in the Supreme Lord alone with loving devotion, O \nArjuna. By His grace you shall attain supreme peace and the Eternal Abode. (18.62) Thus, I have explained the knowledge that is \nmore secret than the secret. After fully reflecting on this, do as you \nwish. (18.63) \nPath of surrender is the ultimate path to God\nHear once again My most secret, supreme word. You are very \ndear to Me; therefore, I shall tell this for your benefit. (18.64) Fix \nyour mind on Me, be devoted to Me, offer service to Me, bow down \nto Me, and you shall certainly reach Me. I promise you, because \nyou are My very dear friend. (18.65) Setting aside doership, ownership and attachment to the fruits of all works, just surrender to—\nor gladly abide by— My Will. I shall liberate you from a

In [60]:
# get the page content
context_page_content = [doc.page_content for doc in context]
# convert that into text and each item is different paragraph
context_page_content_as_text = '\n'.join(context_page_content)

In [62]:
print(context_page_content_as_text)

realization in this very life as if there were no reincarnation. Live as 
though this is your last day on this earth. One cannot achieve 
anything through laziness and procrastination. 
Seek refuge in the Supreme Lord alone with loving devotion, O 
Arjuna. By His grace you shall attain supreme peace and the Eternal Abode. (18.62) Thus, I have explained the knowledge that is 
more secret than the secret. After fully reflecting on this, do as you 
wish. (18.63) 
Path of surrender is the ultimate path to God
Hear once again My most secret, supreme word. You are very 
dear to Me; therefore, I shall tell this for your benefit. (18.64) Fix 
your mind on Me, be devoted to Me, offer service to Me, bow down 
to Me, and you shall certainly reach Me. I promise you, because 
you are My very dear friend. (18.65) Setting aside doership, ownership and attachment to the fruits of all works, just surrender to—
or gladly abide by— My Will. I shall liberate you from all sins or 
bondage of Karma. Do not 